In [1]:
import os
import re
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [2]:
links= {'100':['ap-am', 'am-ap', 'ap-pa', 'pa-ap', 'pa-to', 'to-pa', 'to-df', 'df-to', 'df-mt', 'mt-df', 'mt-ro', 'ro-mt', 'mt-ms', 'ms-mt', 'df-ma', 'ma-df', 'ma-pi', 'pi-ma', 'pi-ba', 'ba-pi', 'df-rj', 'rj-df', 'rj-es', 'es-rj', 'ba-df', 'df-ba', 'pi-ce', 'ce-pi', 'ce-rn', 'rn-ce', 'rn-pb', 'pb-rn', 'pb-pe', 'pe-pb', 'pe-al', 'al-pe', 'al-se', 'se-al', 'se-ba', 'ba-se', 'ce-ba', 'ba-ce', 'ba-mg', 'mg-ba', 'ba-es', 'es-ba', 'mg-sp', 'sp-mg', 'sp-rs', 'rs-sp', 'pr-sc', 'sc-pr', 'rs-sc', 'sc-rs'], '200':['ce-sp', 'sp-ce', 'rj-sp', 'sp-rj', 'pr-rs', 'rs-pr'], '300':['pr-sp', 'sp-pr'], '40':['pb-ba', 'ba-pb'], '20':['go-df', 'df-go', 'df-sp', 'sp-df', 'mg-rj', 'rj-mg'], '6':['ac-ro', 'ro-ac'], '3':['am-df', 'df-am'], '1':['am-rr', 'rr-am', 'rr-ce', 'ce-rr'], '10':['mt-go', 'go-mt', 'ms-go', 'go-ms', 'ms-pr', 'pr-ms', 'df-mg', 'mg-df', 'mg-es', 'es-mg', 'pa-ma', 'ma-pa', 'df-ce', 'ce-df']}

In [3]:
def contar_ocorrencias(row): #aqui eu conto o numero de saltos 
        ocorrencias = row.astype(str).str.contains('bkb.rnp.br|No Hostname|pop').sum() #bkb.rnp.br
        return ocorrencias

def substituir_prefixo(df):
    def processar_celula(valor):
        if isinstance(valor, str):
            if '.bkb.rnp.br' in valor:
                prefixo = valor.split('.')[0].split('-')
                prefixo = prefixo[0] + '-' + prefixo[1]
                prefixo = re.sub(r'\d+|lan|mx', '', prefixo)
                partes = prefixo.split('-')
                if len(partes) > 0 and len(partes[0]) == 3:
                    partes[0] = partes[0][1:] 
                if len(partes) > 1 and len(partes[1]) == 3:
                    partes[1] = partes[1][1:] 
                return '-'.join(partes)
            else:
                return np.nan
        else:
            return np.nan

    colunas_para_processar = df.columns[~df.columns.isin(['Vazao','Vazao_bbr', 'Atraso(ms)', 'Timestamp_cubic', 'Hop_count'])]

    df[colunas_para_processar] = df[colunas_para_processar].map(processar_celula)
    
    return df


def obter_menor_chave(linha, links):
    chaves_encontradas = []
    links_gargalo = []
    
    for link in linha:
        for chave, valores in links.items():
            if link in valores:
                chaves_encontradas.append(int(chave))
                links_gargalo.append(link)

    if chaves_encontradas:
        menor_chave = min(chaves_encontradas)
        indice_menor = chaves_encontradas.index(menor_chave)
        link_gargalo = links_gargalo[indice_menor]
        return menor_chave, link_gargalo
    else:
        return np.nan, np.nan 

def aplicar_bottleneck(df, links):
    retorno = df.apply(lambda linha: obter_menor_chave(linha, links), axis=1, result_type='expand')
    
    if retorno.shape[1] != 2:
        print("Erro: Função obter_menor_chave não está retornando duas colunas para algumas linhas.")
        print("Cabeçalho de retorno para depuração:\n", retorno.head())
        print("Dimensões esperadas: 2 colunas. Dimensões atuais:", retorno.shape[1])
        
        retorno = retorno.reindex(columns=[0, 1]).fillna(np.nan)

    df[['Bottleneck', 'Link_bottleneck']] = retorno
    return df


def selecionar_colunas(df):
    colunas_desejadas = ['Timestamp_cubic', 'Vazao', 'Vazao_bbr', 'Atraso(ms)','Hop_count', 'Bottleneck', 'Link_bottleneck']
    return df[colunas_desejadas]


In [5]:
local_vazaocubic = '../datasets/originals/datasets vazao/cubic/'
local_vazaobbr = '../datasets/originals/datasets vazao/bbr/'
local_traceroute = '../datasets/originals/datasets traceroute/'
local_atraso = '../datasets/originals/datasets atraso/'
local_destino = '../datasets/pos-process/'

if not os.path.exists(local_destino):
    os.makedirs(local_destino)

tolerance_seconds = 6*3600

def merge_with_tolerance(df1, df2, left_on, right_on, tolerance, suffixes=('_x', '_y')):
    return pd.merge_asof(df1, df2, left_on=left_on, right_on=right_on, direction='nearest', tolerance=tolerance, suffixes=suffixes)

for arquivo1 in sorted(os.listdir(local_vazaocubic)):
    
    with open(local_vazaocubic + arquivo1, 'r') as f:
        line_count = sum(1 for line in f) - 1  # Subtrai 1 para ignorar o cabeçalho

    if line_count < 10:
        print(f"O arquivo {arquivo1} é muito pequeno.")
        continue
        

    nome_links1 = re.findall(r"\b\w{2}-\w{2}\b", arquivo1[:50])
    
    df_vazao_cubic = pd.read_csv(local_vazaocubic + arquivo1)
    
    df_vazao_cubic = df_vazao_cubic.rename(columns={'Timestamp': 'Timestamp_cubic'})
    

    merged = df_vazao_cubic.sort_values('Timestamp_cubic')


    bbr_file = next((f for f in os.listdir(local_vazaobbr) if re.findall(r"\b\w{2}-\w{2}\b", f[:50])[0] == nome_links1[0]), None)
    if bbr_file:
        df_vazao_bbr = pd.read_csv(local_vazaobbr + bbr_file)
        df_vazao_bbr = df_vazao_bbr.rename(columns={'Timestamp': 'Timestamp_bbr'})
        df_vazao_bbr = df_vazao_bbr.sort_values('Timestamp_bbr')
        
        merged = merge_with_tolerance(merged, df_vazao_bbr, left_on='Timestamp_cubic', right_on='Timestamp_bbr', tolerance=tolerance_seconds, suffixes=('', '_bbr'))

    traceroute_file = next((f for f in os.listdir(local_traceroute) if re.findall(r"\b\w{2}-\w{2}\b", f[:40])[0] == nome_links1[0]), None)
    if traceroute_file:
        try:
            df_traceroute = pd.read_csv(local_traceroute + traceroute_file, header=None, engine='python', on_bad_lines='skip')
            df_traceroute.columns = ['Timestamp_traceroute'] + list(df_traceroute.columns[1:])
            df_traceroute = df_traceroute.sort_values('Timestamp_traceroute')
            merged = merge_with_tolerance(merged, df_traceroute, left_on='Timestamp_cubic', right_on='Timestamp_traceroute', tolerance=tolerance_seconds)

        except pd.errors.ParserError as e:
            print(f"Erro ao ler o arquivo {traceroute_file}: {e}")
            continue

    atraso_file = next((f for f in os.listdir(local_atraso) if re.findall(r"\b\w{2}-\w{2}\b", f[:50])[0] == nome_links1[0]), None)
    if atraso_file:
        df_atraso = pd.read_csv(local_atraso + atraso_file)
        df_atraso = df_atraso.rename(columns={'Timestamp': 'Timestamp_atraso'})
        df_atraso = df_atraso.sort_values('Timestamp_atraso')
        
        merged = merge_with_tolerance(merged, df_atraso, left_on='Timestamp_cubic', right_on='Timestamp_atraso', tolerance=tolerance_seconds)

    columns_to_check = ['Vazao', 'Vazao_bbr', 'Atraso(ms)', 'Timestamp_cubic']
    merged = merged.dropna(subset=columns_to_check)

    
    merged = merged.drop(columns=['Data_x','Data_y','Timestamp_traceroute','Timestamp_atraso']) #'Timestamp_bbr'
    
    #aqui vou chamar as funcoes de numero de saltos e gargalo do traceroute 
    merged['Hop_count'] = merged.apply(contar_ocorrencias, axis=1)
    # aqui a gente perde os valores das outras colunas -  se quiser o timestamp bbr vamos precisar ajustar isso
    merged = substituir_prefixo(merged)
    merged = aplicar_bottleneck(merged, links)
    merged = selecionar_colunas(merged)
    
    

    columns_to_check = ['Vazao', 'Vazao_bbr', 'Atraso(ms)', 'Timestamp_cubic', 'Hop_count', 'Bottleneck', 'Link_bottleneck']
    merged = merged.dropna(subset=columns_to_check)
    
    
    merged.rename(columns={'Vazao': 'Vazao_cubic'}, inplace=True)
    if not merged.empty:
        output_file = os.path.join(local_destino, f"dataset-completo-{nome_links1[0]}.csv")
        output_file_full_path = os.path.abspath(output_file)
        print(f"Caminho completo do arquivo de saída: {output_file_full_path}")

        try:
            merged.to_csv(output_file, index=False)
            print(f"Arquivo gerado: {output_file}")
        except Exception as e:
            print(f"Erro ao salvar o arquivo {output_file}: {e}")
    else:
        print(f"Nenhum dado correspondente encontrado para: {arquivo1}")

print("Processamento concluído!")


Caminho completo do arquivo de saída: /home/malu/Área de Trabalho/SBRC-REGRESSAO/ESTIMATIVA-REGRESSAO-SBRC-2025/datasets/pos-process/dataset-completo-ac-am.csv
Arquivo gerado: ../datasets/pos-process/dataset-completo-ac-am.csv
Caminho completo do arquivo de saída: /home/malu/Área de Trabalho/SBRC-REGRESSAO/ESTIMATIVA-REGRESSAO-SBRC-2025/datasets/pos-process/dataset-completo-ac-ap.csv
Arquivo gerado: ../datasets/pos-process/dataset-completo-ac-ap.csv
Caminho completo do arquivo de saída: /home/malu/Área de Trabalho/SBRC-REGRESSAO/ESTIMATIVA-REGRESSAO-SBRC-2025/datasets/pos-process/dataset-completo-ac-ba.csv
Arquivo gerado: ../datasets/pos-process/dataset-completo-ac-ba.csv
Caminho completo do arquivo de saída: /home/malu/Área de Trabalho/SBRC-REGRESSAO/ESTIMATIVA-REGRESSAO-SBRC-2025/datasets/pos-process/dataset-completo-ac-ce.csv
Arquivo gerado: ../datasets/pos-process/dataset-completo-ac-ce.csv
O arquivo cubic esmond data ac-df 07-13-2024.csv é muito pequeno.
Caminho completo do arquiv